In [90]:
import os
import subprocess
import sys
# import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint
from PIL import Image 
import pandas as pd
import sqlite3
from IPython.display import IFrame


# Ask GRASS GIS where its Python packages are.
gisbase = subprocess.check_output(["grass", "--config", "path"], text=True).strip()
os.environ["GISBASE"] = gisbase
print(gisbase)

# Ask GRASS GIS where its Python packages are.
sys.path.append(
    subprocess.check_output(["grass", "--config", "python_path"], text=True).strip()
)

# Import the GRASS GIS packages we need.
import grass.script as gs

# Import GRASS Jupyter
import grass.jupyter as gj

/usr/local/grass83


In [91]:
gisdb = os.path.join(os.getenv('HOME'), 'grassdata')
site = 'tx069-playas'
gj.init(gisdb, site, 'basic')

In [92]:
gs.run_command('g.region', raster='elevation', flags='pa')

projection: 1 (UTM)
zone:       13
datum:      wgs84
ellipsoid:  wgs84
north:      3828447.35560415
south:      3811791.35560415
west:       755357.33549331
east:       776005.33549331
nsres:      8
ewres:      8
rows:       2082
cols:       2581
cells:      5373642


In [94]:
vs_map = gj.InteractiveMap()

vs_map.add_raster("elevation", opacity=0.8)
vs_map.add_raster("aspect", opacity=0.5)
vs_map.add_layer_control()
# vs_map.add_query_button()
vs_map.show()

Map(center=[0.0, 0.0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_t…

In [26]:
gs.run_command('r.info', map='aspect') 

 +----------------------------------------------------------------------------+
 | Map:      aspect                         Date: Thu Jun  6 18:25:17 2024    |
 | Mapset:   basic                          Login of Creator: coreywhite      |
 | Project:  tx069-playas                                                     |
 | DataBase: /home/coreywhite/grassdata                                       |
 | Title:    Aspect counterclockwise in degrees from east                     |
 | Timestamp: none                                                            |
 |----------------------------------------------------------------------------|
 |                                                                            |
 |   Type of Map:  raster               Number of Categories: 360             |
 |   Data Type:    FCELL                Semantic label: (none)                |
 |   Rows:         2082                                                       |
 |   Columns:      2581                 